# Métriques de classification


Rappelez-vous que l'exactitude (accuracy) d'un modèle de classification par rapport à un ensemble de données particulier est égale à la proportion d'observations dans l'ensemble de données pour lesquelles le modèle génère des prédictions correctes. 

L'exactitude est une mesure couramment utilisée pour mesurer les performances d'un modèle de classification. Cependant, cela peut parfois donner des résultats trompeurs, notamment lorsque l’on travaille avec des ensembles de données dans lesquels les classes sont très déséquilibrées. Lorsqu’une grande majorité des observations d’un ensemble de données appartiennent à une seule classe, un « modèle naïf » qui prédit toujours la classe majoritaire obtiendra un score d'exactitude très élevé, mais sera totalement inutile. 

Dans cette leçon, nous présenterons deux nouvelles métriques de classification, appelées **précision** et **rappel**. Ces métriques peuvent être utilisées pour nous donner une idée des performances d'un modèle de classification sur des classes spécifiques, plutôt que sur l'ensemble de données dans son ensemble. 

Nous commencerons par importer quelques outils dont nous aurons besoin dans cette leçon. Prenez note des fonctions `classification_report` et `confusion_matrix` que nous importons depuis le module `sklearn.metrics`. Nous discuterons de ces fonctions plus tard dans la leçon.

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

## Générer des données
Pour illustrer les concepts abordés, nous générerons des données synthétiques. Supposons que nous ayons un problème de classification dans lequel les observations sont regroupées dans l'une des trois classes, étiquetées « A », « B » et « C ». 
Dans la cellule ci-dessous, nous générons aléatoirement deux tableaux y_true et y_pred, chacun contenant 1000 éléments. 
Supposons que y_true contienne les classes observées pour les observations dans un ensemble de test qui a été mis de côté, tandis que y_pred contient les classes prédites pour l'ensemble de test, telles que générées par un modèle de classification.

In [4]:
np.random.seed(1)
n = 1000
y_true = np.random.choice(['A', 'B', 'C'],size=n, p=[0.2, 0.5, 0.3])
ix = np.random.choice(range(n), replace=False, size=400)
y_pred = y_true.copy()
y_pred[ix] = np.random.choice(['A', 'B', 'C'], size=400, p=[0.2, 0.5, 0.3])

print('First 10 elements of y_true:', y_true[:10])
print('First 10 elements of y_pred:', y_pred[:10])

First 10 elements of y_true: ['B' 'C' 'A' 'B' 'A' 'A' 'A' 'B' 'B' 'B']
First 10 elements of y_pred: ['B' 'C' 'A' 'B' 'B' 'C' 'A' 'B' 'B' 'C']


Dans la cellule ci-dessous, nous utilisons `y_pred` et `y_true` pour calculer l'exactitude du modèle sur cet ensemble de test.

In [5]:
accuracy = np.mean(y_pred == y_true)
print('Test Accuracy:', accuracy)

Test Accuracy: 0.739


## Matrice de confusion

Une **matrice de confusion** est un tableau qui peut être utilisé pour avoir une idée de la façon dont un modèle de classification fonctionne sur des classes individuelles, ainsi que des types d'erreurs que le modèle est susceptible de commettre. Une matrice de confusion est structurée de manière à comporter une ligne et une colonne pour chaque classe. Les lignes sont associées aux valeurs réelles observées de chaque classe, tandis que les colonnes sont associées aux valeurs prédites de chaque classe. Chaque cellule du tableau contient un nombre, qui correspond au nombre d'observations qui se trouvent réellement dans la classe associée à la ligne dans laquelle se trouve la cellule et qui devraient être membres de la classe associée à la colonne de la cellule. 

Nous pouvons utiliser la fonction `confusion_matrix` de `sklearn.metrics` pour générer automatiquement une matrice de confusion basée sur les tableaux d'étiquettes observés et prédits.

In [6]:
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[123  48  32]
 [ 42 394  64]
 [ 21  54 222]]


La matrice de confusion est renvoyée sous forme de tableau 2D. Pour rendre ces informations plus faciles à lire, nous allons les convertir en DataFrame et ajouter des noms pour les lignes et les colonnes.

In [7]:
pd.DataFrame(cm, index=['Actual A', 'Actual B', 'Actual C'], 
            columns=['Pred A', 'Pred B', 'Pred C'], )

Pred A  Pred B  Pred C
Actual A     123      48      32
Actual B      42     394      64
Actual C      21      54     222

En lisant la première ligne de la matrice de confusion, nous voyons que sur les 203 observations de classe A trouvées dans l'ensemble de test, le modèle prédit que 123 étaient dans la classe A, 48 dans la classe B et 32 ​​dans la classe C. Nous pouvons lire les autres lignes de la même manière.

## Vrai positif, faux positif, vrai négatif, faux négatif

Supposons que nous ayons un problème de classification avec une variable de réponse « Y ». Supposons que nous soyons particulièrement intéressés par les performances d'un modèle par rapport à l'une des classes de « Y ». Nous appellerons cette classe spécifique la **Classe positive**. Nous appellerons toutes les autres classes **Classes négatives**. Pour des raisons de commodité de notation, nous appellerons la classe positive « Classe P ». 


* Une observation est un **Vrai positif** pour la classe « P » si le modèle prédit que l'observation est positive et que la classe observée est réellement positive. 

* Une observation est un **faux positif** pour la classe « C » si le modèle prédit que l'observation est positive, mais que la classe observée est en réalité négative. 

* Une observation est un **Vrai Négatif** pour la Classe « P » si le modèle prédit que l'observation est négative et que la classe observée est en réalité négative. 

* Une observation est un **faux négatif** pour la classe « P » si le modèle prédit que l'observation est négative, mais que la classe observée est en réalité positive. 

Pour une classe particulière, soit « TP », « FP », « TN » et « FN » font référence respectivement au nombre de vrais positifs, de faux positifs, de vrais négatifs et de faux négatifs. 

Dans la cellule ci-dessous, nous calculons `TP`, `FP`, `TN` et `FN` pour chacune des trois classes de notre exemple, puis affichons les résultats sous forme de DataFrame.

In [8]:
ClassA = [np.sum((y_pred == 'A') & (y_true == 'A')), np.sum((y_pred == 'A') & (y_true != 'A')), 
          np.sum((y_pred != 'A') & (y_true != 'A')), np.sum((y_pred != 'A') & (y_true == 'A')), ]

ClassB = [np.sum((y_pred == 'B') & (y_true == 'B')), np.sum((y_pred == 'B') & (y_true != 'B')), 
          np.sum((y_pred != 'B') & (y_true != 'B')), np.sum((y_pred != 'B') & (y_true == 'B')), ]

ClassC = [np.sum((y_pred == 'C') & (y_true == 'C')), np.sum((y_pred == 'C') & (y_true != 'C')), 
          np.sum((y_pred != 'C') & (y_true != 'C')), np.sum((y_pred != 'C') & (y_true == 'C')), ]

pd.DataFrame([ClassA, ClassB, ClassC], 
            index=['Class A', 'Class B', 'Class C'], 
            columns=['TP', 'FP', 'TN', 'FN'])

TP   FP   TN   FN
Class A  123   63  734   80
Class B  394  102  398  106
Class C  222   96  607   75

## Précision et rappel

Nous définissons la **précision** et le **rappel** d'un modèle par rapport à une classe particulière dans un ensemble de données comme suit : 

* **Précision** $\large = \frac{TP}{TP ~+~ FP} ~=~ \frac{TP}{\textrm{Nombre de prédictions de classe positive} }$ 

* **Rappel** $\large =\frac{TP}{TP ~+~ FN} ~=~ \frac{TP}{\textrm{Nombre d'observations positives réelles} }$
  
La précision peut être considérée comme une estimation de la probabilité qu’une prédiction positive soit réellement correcte. Le rappel peut être interprété comme une estimation de la probabilité qu'une observation positive soit correctement identifiée par le modèle.


In [12]:
classA_precision = np.sum((y_pred == 'A') & (y_true == 'A') ) / np.sum(y_pred == 'A')
classA_recall = np.sum((y_pred == 'A') & (y_true == 'A') ) / np.sum(y_true == 'A')

print('Class A Precision:', round(classA_precision,4))
print('Class A Recall:   ', round(classA_recall,4))

classB_precision = np.sum((y_pred == 'B') & (y_true == 'B') ) / np.sum(y_pred == 'B')
classB_recall = np.sum((y_pred == 'B') & (y_true == 'B') ) / np.sum(y_true == 'B')

print('Class B Precision:', round(classB_precision,4))
print('Class B Recall:   ', round(classB_recall,4))

Class A Precision: 0.6613
Class A Recall:    0.6059
Class B Precision: 0.7944
Class B Recall:    0.788


Le score de précision suggère que lorsque notre modèle prédit qu'une observation appartient à la classe A, elle sera correcte environ 66,13 % du temps. 
Le score de rappel suggère que notre modèle identifiera correctement environ 60,59 % de toutes les observations qui appartiennent réellement à la classe A.

## Rapport de classification

La fonction `classification_report` de `sklearn.metrics` peut être utilisée pour calculer automatiquement les scores de précision et de rappel pour chaque classe. Nous afficherons ces résultats pour notre exemple ci-dessous.

In [13]:
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           A     0.6613    0.6059    0.6324       203
           B     0.7944    0.7880    0.7912       500
           C     0.6981    0.7475    0.7220       297

    accuracy                         0.7390      1000
   macro avg     0.7179    0.7138    0.7152      1000
weighted avg     0.7388    0.7390    0.7384      1000

